In [3]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
import seaborn as sns

# from tensorflow.keras.utils import np_utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input, EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, Conv2D

from sklearn.model_selection import train_test_split

import tensorflow as tf
print(tf.__version__)

In [4]:
fpath = '/kaggle/input/stanford-dogs-dataset/images/Images/'
categories = os.listdir(fpath)

In [5]:
imgsize = 224
batch_size=64
n_classes = 120

# for index, folder in enumerate(categories[:n_classes]):
#     path = os.path.join(fpath,folder)
#     for img in tqdm(os.listdir(path)):
#         Ipath = os.path.join(path,img)
#         img = cv2.imread(Ipath,cv2.IMREAD_COLOR)
#         img = cv2.resize(img,(imgsize,imgsize))
#         X.append(np.array(img))
#         Y.append(index)

# X = np.array(X).astype('float16')
# Y = np.array(Y)
# print(X.shape)
# print(Y.shape)

In [6]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

ds_train = image_dataset_from_directory(fpath,
                                       labels='inferred',
                                       label_mode='int',
                                       color_mode="rgb",
                                       batch_size=batch_size,
                                       image_size=(imgsize, imgsize),
                                       shuffle=True,
                                       seed=42,
                                       validation_split=0.2,
                                       subset='training',
                                       )

ds_validation = image_dataset_from_directory(fpath,
                                       labels='inferred',
                                       label_mode='int',
                                       color_mode="rgb",
                                       batch_size=batch_size,
                                       image_size=(imgsize, imgsize),
                                       shuffle=True,
                                       seed=42,
                                       validation_split=0.2,
                                       subset='validation',
                                       )

In [7]:
# augs_gen = ImageDataGenerator(preprocessing_function=preprocess_input,featurewise_center=False,  
#         samplewise_center=False, 
#         featurewise_std_normalization=False,  
#         samplewise_std_normalization=False,  
#         zca_whitening=False,  
#         rotation_range=10,  
#         zoom_range = 0.1, 
#         width_shift_range=0.2,  
#         height_shift_range=0.2, 
#         horizontal_flip=True,  
#         vertical_flip=False)

# augs_gen.fit(trainImg)

In [8]:
##modelling
base_model = EfficientNetB0(include_top=False,
                  input_shape = (imgsize,imgsize,3),
                  weights = 'imagenet')

base_model.trainable = False

model = Sequential([base_model])
#model.add(Conv2D(10, (3,3) , activation = 'relu'))
model.add(GlobalAveragePooling2D())
#model.add(Dropout(0.5))
#model.add(Dense(100,activation='relu'))
model.add(Dense(n_classes,activation='softmax'))
model.summary()

metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy"), 
          tf.keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy")]
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=metrics
)


In [9]:
ES = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 3, restore_best_weights=True)

In [10]:
h = model.fit(ds_train, epochs = 40, verbose = 1, 
              batch_size = batch_size, validation_data  = ds_validation, 
              callbacks = [ES])

In [11]:
plt.figure(figsize = (10,5))
plt.subplot(1,2,1)
plt.plot(h.history['accuracy'],     label = 'Train')
plt.plot(h.history['val_accuracy'], label = 'Validation')
plt.title('Accuracy')
plt.legend(loc = 'upper left')

plt.subplot(1,2,2)
plt.plot(h.history['loss']    ,label = 'Train')
plt.plot(h.history['val_loss'],label = 'Validation')
plt.title('Loss')
plt.legend(loc = 'upper left')
plt.show()

max_val_acc = 100*np.max(h.history['val_accuracy'])
print("Best Validation Accuracy: {}%".format(max_val_acc))

In [12]:
tf.keras.models.save_model(model, filepath='EfficientB0_trained_90.h5')